In [56]:
import shutil
shutil.rmtree('/content/sample_data', ignore_errors=True)

# YOLOv5s

In [57]:
import os
import sys
import subprocess
import urllib.request
import zipfile
from pathlib import Path

Install required packages

In [58]:
def install_requirements():
    packages = [
        "ultralytics",
        "torch",
        "torchvision",
        "tensorflow",
        "onnx",
        "onnx2tf",
        "onnxsim",
        "opencv-python",
    ]

    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    print("All required packages installed successfully!")

Check for existing dataset or download COCO128

In [59]:
def prepare_dataset():
    # List of common dataset locations to check
    potential_datasets = [
        "/content/dataset",
        "/content/custom_dataset",
        "/content/yolo_dataset",
        "/content/coco128"
    ]

    # Check for existing datasets
    for dataset_path in potential_datasets:
        if os.path.exists(dataset_path):
            # Look for YOLO dataset structure (images and labels folders)
            images_dir = os.path.join(dataset_path, 'images')
            labels_dir = os.path.join(dataset_path, 'labels')

            # Also check for common YOLO yaml files
            yaml_files = [f for f in os.listdir(dataset_path) if f.endswith('.yaml') or f.endswith('.yml')]

            if os.path.exists(images_dir) and os.path.exists(labels_dir):
                # Check if folders have content
                has_images = len([f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]) > 0
                has_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith('.txt')]) > 0

                if has_images and has_labels:
                    print(f"Existing dataset found at: {dataset_path}")

                    # Try to find or create a yaml file
                    yaml_path = None
                    if yaml_files:
                        yaml_path = os.path.join(dataset_path, yaml_files[0])
                        print(f"Using existing yaml file: {yaml_files[0]}")
                    else:
                        # Create a basic yaml file
                        yaml_path = create_basic_yaml(dataset_path)

                    return dataset_path, yaml_path

    # No existing dataset found, download COCO128
    print("No existing dataset found. Downloading COCO128...")
    coco128_path = download_coco128()
    yaml_path = os.path.join(coco128_path, 'coco128.yaml')

    return coco128_path, yaml_path

In [60]:
def create_basic_yaml(dataset_path):
    yaml_content = f"""# Dataset configuration
path: {dataset_path}
train: images
val: images

# Classes (will be auto-detected from labels)
nc: 80  # number of classes (placeholder)
names: ['class0', 'class1', 'class2']  # class names (placeholder)
"""
    yaml_path = os.path.join(dataset_path, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"Created basic yaml configuration: {yaml_path}")
    return yaml_path

In [61]:
def download_coco128():
    dataset_url = "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip"
    dataset_path = "/content/coco128.zip"
    extract_path = "/content/"

    print("Downloading COCO128 dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_path)

    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Remove zip file to save space
    os.remove(dataset_path)

    # Verify the yaml file exists and fix path if needed
    coco128_dir = "/content/coco128"
    yaml_file = os.path.join(coco128_dir, "coco128.yaml")

    if not os.path.exists(yaml_file):
        print("coco128.yaml not found, creating configuration...")
        create_coco128_yaml(coco128_dir)

    print("COCO128 dataset downloaded and extracted successfully!")
    return coco128_dir

In [62]:
def create_coco128_yaml(dataset_path):
    yaml_content = f"""# COCO128 dataset configuration
path: {dataset_path}
train: images/train2017
val: images/train2017

# Classes
nc: 80
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']
"""

    yaml_path = os.path.join(dataset_path, 'coco128.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"Created coco128.yaml at: {yaml_path}")

In [63]:
def train_yolov5s(yaml_path, epochs=100, imgsz=640):
    from ultralytics import YOLO

    print("Initializing YOLOv5s model...")
    model = YOLO('yolov5s.pt')  # Load pretrained YOLOv5s model

    print(f"Starting training for {epochs} epochs...")
    print(f"Using dataset configuration: {yaml_path}")

    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        device=0 if os.system('nvidia-smi') == 0 else 'cpu',  # Use GPU if available
        project='/content/yolov5_training',
        name='yolov5s_experiment',
        save=True,
        cache=True,
        verbose=True,
        patience=10,  # Early stopping patience
        save_period=10  # Save checkpoint every 10 epochs
    )

    print("Training completed!")
    return model, results

In [64]:
def export_model_formats(model, export_dir="/content/exported_models"):
    os.makedirs(export_dir, exist_ok=True)

    print("Exporting YOLOv5s model to different formats...")

    # Export to ONNX (intermediate format for TensorFlow conversion)
    print("Exporting to ONNX...")
    onnx_path = model.export(format='onnx', dynamic=False, simplify=True)
    print(f"ONNX model exported to: {onnx_path}")

    # Export to TorchScript
    print("Exporting to TorchScript...")
    try:
        torchscript_path = model.export(format='torchscript')
        print(f"TorchScript model exported to: {torchscript_path}")
    except Exception as e:
        print(f"TorchScript export failed: {e}")
        torchscript_path = None

    # Export to TensorFlow SavedModel format
    print("Exporting to TensorFlow SavedModel...")
    try:
        tf_path = model.export(format='saved_model')
        print(f"TensorFlow SavedModel exported to: {tf_path}")
    except Exception as e:
        print(f"Direct TensorFlow export failed: {e}")
        tf_path = None

    return onnx_path, tf_path, torchscript_path

In [65]:
def convert_to_tflite(onnx_path, output_dir="/content/tflite_models"):
    import tensorflow as tf

    os.makedirs(output_dir, exist_ok=True)

    try:
        # First convert ONNX to TensorFlow SavedModel using onnx2tf
        print("Converting ONNX to TensorFlow SavedModel...")
        saved_model_dir = f"{output_dir}/saved_model"

        # Use onnx2tf for conversion
        cmd = f"onnx2tf -i {onnx_path} -o {saved_model_dir} --non_verbose"
        subprocess.run(cmd, shell=True, check=True)

        # Convert TensorFlow SavedModel to TensorFlow Lite
        print("Converting TensorFlow SavedModel to TensorFlow Lite...")
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

        # Optimization settings for mobile deployment
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]  # Use FP16 for smaller size

        # Additional optimization for YOLOv5s
        converter.allow_custom_ops = True
        converter.experimental_new_converter = True

        # Convert the model
        tflite_model = converter.convert()

        # Save the TensorFlow Lite model
        tflite_path = f"{output_dir}/yolov5s_coco128.tflite"
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)

        print(f"TensorFlow Lite model saved to: {tflite_path}")

        # Get model size
        model_size = os.path.getsize(tflite_path) / (1024 * 1024)  # Size in MB
        print(f"TensorFlow Lite model size: {model_size:.2f} MB")

        return tflite_path

    except Exception as e:
        print(f"TensorFlow Lite conversion failed: {e}")
        print("Trying quantized conversion method...")

        # Try quantized conversion for better compatibility
        try:
            return convert_to_tflite_quantized(onnx_path, output_dir)
        except Exception as e2:
            print(f"Quantized conversion also failed: {e2}")
            return None

In [66]:
def convert_to_tflite_quantized(onnx_path, output_dir):
    import tensorflow as tf

    print("Attempting quantized TensorFlow Lite conversion...")

    saved_model_dir = f"{output_dir}/saved_model"

    # Convert ONNX to SavedModel first
    cmd = f"onnx2tf -i {onnx_path} -o {saved_model_dir} --non_verbose"
    subprocess.run(cmd, shell=True, check=True)

    # Convert with quantization
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

    # Enable quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = generate_representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8

    # Convert
    quantized_tflite_model = converter.convert()

    # Save quantized model
    quantized_tflite_path = f"{output_dir}/yolov5s_quantized.tflite"
    with open(quantized_tflite_path, 'wb') as f:
        f.write(quantized_tflite_model)

    model_size = os.path.getsize(quantized_tflite_path) / (1024 * 1024)
    print(f"Quantized TensorFlow Lite model saved to: {quantized_tflite_path}")
    print(f"Quantized model size: {model_size:.2f} MB")

    return quantized_tflite_path

In [67]:
def generate_representative_dataset():
    import numpy as np

    # Generate representative dataset for quantization
    for _ in range(100):
        # Random input matching YOLOv5s input shape [1, 3, 640, 640]
        yield [np.random.uniform(0.0, 1.0, size=(1, 3, 640, 640)).astype(np.float32)]

In [68]:
def validate_tflite_model(tflite_path, test_image_path=None):
    import tensorflow as tf
    import numpy as np

    # Load TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("TensorFlow Lite Model Validation:")
    print(f"Model path: {tflite_path}")
    print(f"Input shape: {input_details[0]['shape']}")
    print(f"Input type: {input_details[0]['dtype']}")
    print(f"Number of outputs: {len(output_details)}")

    for i, output in enumerate(output_details):
        print(f"Output {i} shape: {output['shape']}")
        print(f"Output {i} type: {output['dtype']}")

    # Test with random input if no test image provided
    if test_image_path is None:
        input_shape = input_details[0]['shape']
        test_input = np.random.random(input_shape).astype(input_details[0]['dtype'])

        # Normalize input to [0, 1] range if float32
        if input_details[0]['dtype'] == np.float32:
            test_input = test_input.astype(np.float32)
        elif input_details[0]['dtype'] == np.uint8:
            test_input = (test_input * 255).astype(np.uint8)

        interpreter.set_tensor(input_details[0]['index'], test_input)
        interpreter.invoke()

        print("Test inference completed successfully!")

        # Get outputs
        for i, output in enumerate(output_details):
            output_data = interpreter.get_tensor(output['index'])
            print(f"Output {i} shape: {output_data.shape}")
            print(f"Output {i} min/max: {output_data.min():.4f}/{output_data.max():.4f}")

    return True

In [69]:
def benchmark_model(tflite_path, num_runs=50):
    import tensorflow as tf
    import numpy as np
    import time

    print(f"Benchmarking model: {tflite_path}")

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    input_shape = input_details[0]['shape']
    input_dtype = input_details[0]['dtype']

    # Prepare test input
    if input_dtype == np.float32:
        test_input = np.random.random(input_shape).astype(np.float32)
    else:
        test_input = np.random.randint(0, 256, size=input_shape, dtype=input_dtype)

    # Warm up
    for _ in range(5):
        interpreter.set_tensor(input_details[0]['index'], test_input)
        interpreter.invoke()

    # Benchmark
    times = []
    for _ in range(num_runs):
        start_time = time.time()
        interpreter.set_tensor(input_details[0]['index'], test_input)
        interpreter.invoke()
        end_time = time.time()
        times.append((end_time - start_time) * 1000)  # Convert to ms

    avg_time = np.mean(times)
    std_time = np.std(times)

    print(f"Average inference time: {avg_time:.2f} ± {std_time:.2f} ms")
    print(f"FPS: {1000/avg_time:.1f}")

    return avg_time

In [70]:
def main():
    print("=== YOLOv5s Training and TensorFlow Lite Conversion Pipeline ===")
    try:
        # Step 1: Install requirements
        print("\n1. Installing required packages...")
        install_requirements()

        # Then run quick test
        print("\nRunning quick inference test...")
        quick_inference_test()

        # Step 2: Check for existing dataset or prepare COCO128
        print("\n2. Checking for existing dataset...")
        dataset_path, yaml_path = prepare_dataset()

        # Step 3: Train model
        print("\n3. Training YOLOv5s model...")
        model, results = train_yolov5s(yaml_path, epochs=50)  # Reduced for faster training

        # Step 4: Export model formats
        print("\n4. Exporting model to different formats...")
        onnx_path, tf_path, torchscript_path = export_model_formats(model)

        # Step 5: Convert to TensorFlow Lite
        print("\n5. Converting to TensorFlow Lite...")
        tflite_path = convert_to_tflite(onnx_path)

        if tflite_path:
            # Step 6: Validate TFLite model
            print("\n6. Validating TensorFlow Lite model...")
            validate_tflite_model(tflite_path)

            # Step 7: Benchmark model
            print("\n7. Benchmarking TensorFlow Lite model...")
            benchmark_model(tflite_path)

            print("\n=== Pipeline Completed Successfully ===")
            print(f"Final TensorFlow Lite model: {tflite_path}")
        else:
            print("\n=== Pipeline Completed with Errors ===")
            print("TensorFlow Lite conversion failed. Check the ONNX model output.")

    except Exception as e:
        print(f"\nPipeline failed with error: {e}")
        import traceback
        traceback.print_exc()

In [71]:
def simple_tflite_conversion():
    from ultralytics import YOLO

    print("Running simplified TensorFlow Lite conversion...")

    # Load pre-trained YOLOv5s
    model = YOLO('yolov5s.pt')

    # Direct export to TensorFlow Lite (if supported)
    try:
        tflite_path = model.export(format='tflite', imgsz=640)
        print(f"Direct TFLite export successful: {tflite_path}")
        return tflite_path
    except Exception as e:
        print(f"Direct TFLite export failed: {e}")
        return None

In [72]:
def quick_inference_test():
    """Quick test to verify YOLOv5s is working"""
    from ultralytics import YOLO
    import numpy as np

    print("Running quick YOLOv5s inference test...")

    # Load model
    model = YOLO('yolov5s.pt')

    # Create dummy image
    dummy_image = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

    # Run inference
    results = model(dummy_image)

    print("YOLOv5s inference test completed successfully!")
    print(f"Number of detections: {len(results[0].boxes) if results[0].boxes is not None else 0}")

In [73]:
if __name__ == "__main__":
    # Run in Google Colab environment
    print("Starting YOLOv5s training and TensorFlow Lite conversion...")
    print("Note: This process may take 1-2 hours depending on training epochs and hardware.")

    # Uncomment the line below to run the full pipeline
    main()

    # For quick testing, uncomment the line below
    simple_tflite_conversion()

Starting YOLOv5s training and TensorFlow Lite conversion...
Note: This process may take 1-2 hours depending on training epochs and hardware.
=== YOLOv5s Training and TensorFlow Lite Conversion Pipeline ===

1. Installing required packages...
All required packages installed successfully!

Running quick inference test...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Running quick YOLOv5s inference test...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 640x640 (no detections), 15.0ms
Speed: 22.7ms preprocess, 15.0ms in

In [74]:
# Create zip of entire content folder
print("Creating backup of content folder...")
!zip -r /content/colab_content.zip /content/ -x "*.zip" "*/.*"

Creating backup of content folder...
  adding: content/ (stored 0%)
  adding: content/yolov5_training/ (stored 0%)
  adding: content/yolov5_training/yolov5s_experiment/ (stored 0%)
  adding: content/yolov5_training/yolov5s_experiment/labels.jpg (deflated 24%)
  adding: content/yolov5_training/yolov5s_experiment/train_batch320.jpg (deflated 9%)
  adding: content/yolov5_training/yolov5s_experiment/val_batch0_labels.jpg (deflated 8%)
  adding: content/yolov5_training/yolov5s_experiment/val_batch2_labels.jpg (deflated 10%)
  adding: content/yolov5_training/yolov5s_experiment/train_batch0.jpg (deflated 3%)
  adding: content/yolov5_training/yolov5s_experiment/BoxP_curve.png (deflated 10%)
  adding: content/yolov5_training/yolov5s_experiment/train_batch321.jpg (deflated 9%)
  adding: content/yolov5_training/yolov5s_experiment/confusion_matrix.png (deflated 21%)
  adding: content/yolov5_training/yolov5s_experiment/train_batch1.jpg (deflated 3%)
  adding: content/yolov5_training/yolov5s_experim